# 03_funcpp03

这是功能像预处理的第三部分。第二部分已经完成了最主要的预处理工作，这部分我们主要是做一些检查和清理工作

# Step 1. 

首先，我们跑一个GLM，把所有的头动信息都回归掉，然后来计算一系列的诊断信息

In [ ]:
# ================================ regress =================================
# compute de-meaned motion parameters (for use in regression)
# in regress (detrend step)

# ------------------------------
# run the regression analysis. Note that stats results should be written as afni format

! 3dDeconvolve -input pb07.{subj}.r*.scale+tlrc.HEAD \
    -mask mask_epi_anat.{subj}.nii.gz \
    -censor motion_{subj}_censor.1D \
    -ortvec motion_demean.1D mot_demean \
    -polort 2 \
    -num_stimts 0 \
    -rout \
    -fout -tout -x1D X.xmat.1D -xjpeg X.jpg \
    -x1D_uncensored X.nocensor.xmat.1D \
    -fitts motpoly.fitts.{subj} \
    -errts motpoly.errts.{subj} \
    -bucket motpoly.stats.{subj}

# should add save a script here

# -- use 3dTproject to project out regression matrix --
! 3dTproject -polort 0 -input pb07.{subj}.r*.scale+tlrc.HEAD \
           -censor motion_{subj}_censor.1D -cenmode ZERO \
           -ort X.nocensor.xmat.1D -prefix motpoly.errts.{subj}.tproject
# 3dTproject can also supply '-passband 0.01, 0.08' to band pass time seriest

! rm -f motpoly.stats* motpoly.fitts* # remove some large files

## Step 2. 计算temporal signal-to-noise ratio

In [ ]:
# --------------------------------------------------
# create a temporal signal to noise ratio dataset
#    signal: if 'scale' block, mean should be 100
#    noise : compute standard deviation of errts
unix_wrapper(f'3dTstat -mean -prefix rm.signal.all.nii.gz all_runs.{subj}.nii.gz"[{ktrs}]"')
unix_wrapper(f'3dTstat -stdev -prefix rm.noise.all.nii.gz motpoly.errts.{subj}.tproject+tlrc"[{ktrs}]"')
! 3dcalc -a rm.signal.all.nii.gz \
       -b rm.noise.all.nii.gz \
       -c full_mask.{subj}.nii.gz \
       -expr "c*a/b" -prefix TSNR.{subj}.nii.gz

# remove redundant files
! rm rm.signal.all* rm.noise.all*
! rm motpoly.errts* X*

## Step 3. clean up

In [ ]:
# ========================== auto block: finalize ==========================
# Remove temporary files
rm -f rm.* # finally remove some left-over files

# We can further remove despike, tshift and volreg data at the end volreg are big
! rm -f all_runs*
! rm anatQQ* MNI152_2009_template_SSW.nii.gz
! rm pb02* pb04* pb05* pb06*
! rm epi_mean_tmp*

# organize data


# ==================== preprocessing done ==================================
# return to previous directory
os.chdir(cwd)
os.environ['SHELL'] = orig_shell     # switch back to the original shell
print(f'\n=============== Preprocessing started: {start_time} ================\n')
print(f'\n=============== Preprocessing finished: {gettimestr("full")} ================\n')